# General E(2)-Equivariant Steerable CNNs  -  A concrete example


In [1]:
import os
import csv
import time
import random
from tqdm import tqdm
import torch

from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation, Pad, Resize, ToTensor, Compose

import numpy as np
from PIL import Image
import cv2

from e2cnn import gspaces
from e2cnn import nn

Finally, we build a **Steerable CNN** and try it MNIST.

Let's also use a group a bit larger: we now build a model equivariant to $8$ rotations.
We indicate the group of $N$ discrete rotations as $C_N$, i.e. the **cyclic group** of order $N$.
In this case, we will use $C_8$.

Because the inputs are still gray-scale images, the input type of the model is again a *scalar field*.

However, internally we use *regular fields*: this is equivalent to a *group-equivariant convolutional neural network*.

Finally, we build *invariant* features for the final classification task by pooling over the group using *Group Pooling*.

The final classification is performed by a two fully connected layers.

# The model

Here is the definition of our model:

In [2]:
class C8SteerableCNN(torch.nn.Module):
    
    def __init__(self, n_classes=10):
        
        super(C8SteerableCNN, self).__init__()
        
        # the model is equivariant under rotations by 45 degrees, modelled by C8
        self.r2_act = gspaces.Rot2dOnR2(N=8)
        #print("self.r2_act", self.r2_act)
        
        # the input image is a scalar field, corresponding to the trivial representation
        in_type = nn.FieldType(self.r2_act, 3*[self.r2_act.trivial_repr])
        #print("in_type", in_type)
        
        # we store the input type for wrapping the images into a geometric tensor during the forward pass
        self.input_type = in_type
        
        # convolution 1
        # first specify the output type of the convolutional layer
        # we choose 24 feature fields, each transforming under the regular representation of C8
        out_type = nn.FieldType(self.r2_act, 24*[self.r2_act.regular_repr])
        #print("out_type", out_type)
        self.block1 = nn.SequentialModule(
            nn.MaskModule(in_type, 80, margin=1),
            nn.R2Conv(in_type, out_type, kernel_size=7, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block1', self.block1.out_type.size)
        
        # convolution 2
        # the old output type is the input type to the next layer
        in_type = self.block1.out_type
        # the output type of the second convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block2 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block2', self.block2.out_type.size)
        self.pool1 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )
        #print('pool1', self.pool1.out_type.size)
        
        # convolution 3
        # the old output type is the input type to the next layer
        in_type = self.block2.out_type
        # the output type of the third convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block3 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block3', self.block3.out_type.size)
        
        # convolution 4
        # the old output type is the input type to the next layer
        in_type = self.block3.out_type
        # the output type of the fourth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block4 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block4', self.block4.out_type.size)
        self.pool2 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )
        #print('pool2', self.pool2.out_type.size)
        
        # convolution 5
        # the old output type is the input type to the next layer
        in_type = self.block4.out_type
        # the output type of the fifth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block5 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block5', self.block5.out_type.size)
        
        # convolution 6
        # the old output type is the input type to the next layer
        in_type = self.block5.out_type
        # the output type of the sixth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 64*[self.r2_act.regular_repr])
        self.block6 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        #print('block6', self.block6.out_type.size)
        self.pool3 = nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=1, padding=0)
        #print('pool3', self.pool3.out_type.size)
        
        self.gpool = nn.GroupPooling(out_type) # pool3.out_type
        
        # number of output channels
        c = self.gpool.out_type.size
        print('gpool', c)
        
        # Fully Connected
        self.fully_net = torch.nn.Sequential(
            torch.nn.Linear(c*13*13, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ELU(inplace=True),
            torch.nn.Linear(64, n_classes),
        )
    
    def forward(self, input: torch.Tensor):
        # wrap the input tensor in a GeometricTensor
        # (associate it with the input type)
        #print("input.shape", input.shape)
        x = nn.GeometricTensor(input, self.input_type)
        
        # apply each equivariant block
        
        # Each layer has an input and an output type
        # A layer takes a GeometricTensor in input.
        # This tensor needs to be associated with the same representation of the layer's input type
        #
        # The Layer outputs a new GeometricTensor, associated with the layer's output type.
        # As a result, consecutive layers need to have matching input/output types
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool1(x)
        
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)
        
        x = self.block5(x)
        x = self.block6(x)
        
        # pool over the spatial dimensions
        x = self.pool3(x)
        #print("x.shape1", x.shape)
        
        # pool over the group
        x = self.gpool(x)
        #print("x.shape2", x.shape)

        # unwrap the output GeometricTensor
        # (take the Pytorch tensor and discard the associated representation)
        x = x.tensor
        #print("x.shape3", x.shape)
        
        # classify with the final fully connected layers)
        x = self.fully_net(x.reshape(x.shape[0], -1)) # x.shape[1]?
        
        return x

Let's try the model on *rotated* MNIST

In [3]:
# # download the dataset
# !wget -nc http://www.iro.umontreal.ca/~lisa/icml2007data/mnist_rotation_new.zip
# # uncompress the zip file
# !unzip -n mnist_rotation_new.zip -d mnist_rotation_new

In [4]:
# import requests
# url = 'http://www.iro.umontreal.ca/~lisa/icml2007data/mnist_rotation_new.zip'
# doc = requests.get(url)
# with open('mnistrot.zip', 'wb') as f:
#     f.write(doc.content)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


Build the dataset

In [6]:
class DOTARotDataset(Dataset):
    
    def __init__(self, mode, transform=None, max_num_examples=0):
        assert mode in ['train', 'test']
        
        basedir = 'C:/Users/Admin/Desktop/data/DOTAv1.0/'
        
        if mode == "train":
            file = basedir+"chips_train/"
        else:
            file = basedir+"chips_val/"
        
        self.transform = transform
        self.max_num_examples = max_num_examples
        self.images = []
        self.labels = []
        new_image_width = 80
        new_image_height = 80
        color = (0,0,0)
        self.classdict = {}
        self.classval = 0

        #data = np.loadtxt(file, delimiter=' ')
        data = []
        
        for root, dirs, filenames in os.walk(file, topdown=False):
            pass
        for d in dirs:
            for root, dirs, filenames in os.walk(file+d, topdown=False):
                pass
            #print(d, len(filenames))
            for f in filenames:
                imgstr = file+d+"/"+f
                #img = Image.open(imgstr)
                #arr = np.asarray(img)
                img = cv2.imread(imgstr)
                old_image_height, old_image_width, channels = img.shape
                #print('old', old_image_height, old_image_width)
                if img.shape[0] < img.shape[1]:
                    result1 = np.full((img.shape[1],img.shape[1], channels), color, dtype=np.uint8)
                    #print('shape', result1.shape)
                    # compute center offset
                    #x_center = np.abs(new_image_width - old_image_width) // 2
                    y_center = np.abs(img.shape[1] - old_image_height) // 2
                    #print('y_center', y_center)
                    # copy img image into center of result image
                    result1[y_center:y_center+old_image_height, :] = img
                elif img.shape[0] > img.shape[1]:
                    result1 = np.full((img.shape[0],img.shape[0], channels), color, dtype=np.uint8)
                    #print('shape', result1.shape)
                    # compute center offset
                    x_center = np.abs(img.shape[0] - old_image_width) // 2
                    #y_center = np.abs(new_image_height - old_image_height) // 2
                    #print('x_center', x_center)
                    # copy img image into center of result image
                    result1[:, x_center:x_center+old_image_width] = img
                else:
                    result1 = img[:,:,:]
                # check if we need to resize
                if not (result1.shape[0] == 80 and result1.shape[1] == 80):
                    #print('result1.shape1', result1.shape)
                    result1 = cv2.resize(result1, dsize=(80, 80), interpolation=cv2.INTER_CUBIC)
                    #print('result1.shape2', result1.shape)
                res2 = result1.reshape(-1, 80, 80).astype(np.float32) # = arr[:, :-1].reshape(-1, 80, 80)
                #print('res2.shape', res2.shape)
                
                # ToTensor screws up the order, so we have to undo it:
                # https://discuss.pytorch.org/t/torchvision-totensor-dont-change-channel-order/82038/2
                #res2 = res2.permute((1, 2, 0)).contiguous()
                
                # convert back to PIL Image object for pytorch transforms (e.g. RandomRotation) to work
                #image = Image.fromarray(image)
                
                self.images.append(res2)
        
                # labels
                if self.classdict.get(d, -1) >= 0:
                    self.labels.append(self.classdict[d])
                else: # it's not in self.classdict yet
                    self.classdict[d] = self.classval
                    self.classval += 1
        
        if self.max_num_examples > 0:
            #z1 = list(zip(self.images, self.labels))
            classdict = {}
            for i in range(len(self.labels)):
                if classdict.get(self.labels[i], False):
                    classdict[self.labels[i]].append(self.images[i])
                else:
                    classdict[self.labels[i]] = [self.images[i]]
            self.images = []
            self.labels = []
            for i in range(self.classval):
                if len(classdict[i]) > self.max_num_examples:
                    temp = random.sample(classdict[i], self.max_num_examples)
                    for t in temp:
                        self.images.append(t)
                        self.labels.append(i)
                else:
                    for t in classdict[i]:
                        self.images.append(t)
                        self.labels.append(i)

        self.num_samples = len(self.labels)
        print("self.num_samples", self.num_samples)
    
    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]
        # image is a numpy ndarray instead of PIL Image object
        # NOTE: certain pytorch functions (aka RandomRotate) require PIL Image objects
        # ToTensor screws up the shape/channel order, so we have to undo it:
        # https://discuss.pytorch.org/t/torchvision-totensor-dont-change-channel-order/82038/2
        #image = image.permute((1, 2, 0)).contiguous()
        #image = Image.fromarray(image)
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.labels)

# images are padded to have shape 29x29.
# this allows to use odd-size filters with stride 2 when downsampling a feature map in the model
#pad = Pad((0, 0, 1, 1), fill=0)

# to reduce interpolation artifacts (e.g. when testing the model on rotated images),
# we upsample an image by a factor of 3, rotate it and finally downsample it again
#resize1 = Resize(80*3)
#resize2 = Resize(80)

#totensor = ToTensor()

Let's build the model

In [7]:
model = C8SteerableCNN(n_classes=15).to(device)

C:\Users\Admin\Desktop\school\e2cnn_experiments\e2cnn\nn\modules\r2_conv\basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:30.)
  full_mask[mask] = norms.to(torch.uint8)


gpool 64


In [8]:
from torchsummary import summary
summary(model, input_size=(3, 80, 80))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
        MaskModule-1            [-1, 3, 80, 80]               0
SingleBlockBasisExpansion-2             [-1, 8, 1, 49]               0
BlocksBasisExpansion-3                [-1, 3, 49]               0
            R2Conv-4          [-1, 192, 76, 76]               0
       BatchNorm3d-5        [-1, 24, 8, 76, 76]              48
    InnerBatchNorm-6          [-1, 192, 76, 76]               0
              ReLU-7          [-1, 192, 76, 76]               0
  SequentialModule-8          [-1, 192, 76, 76]               0
SingleBlockBasisExpansion-9             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-10             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-11             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-12             [-1, 8, 8, 25]               0
SingleBlockBasisExpansion-13             [-1, 8, 8, 25]        

The model is now randomly initialized. 
Therefore, we do not expect it to produce the right class probabilities.

However, the model should still produce the same output for rotated versions of the same image.
This is true for rotations by multiples of $\frac{\pi}{2}$, but is only approximate for rotations by $\frac{\pi}{4}$.

Let's test it on a random test image:
we feed eight rotated versions of the first image in the test set and print the output logits of the model for each of them.

In [9]:
# build the test set
#raw_mnist_test = DOTARotDataset(mode='test')

In [10]:
# retrieve the first image from the test set
#x, y = next(iter(raw_mnist_test))
#
#print(x.shape)
#print(raw_mnist_test[3530])

In [11]:
def test_model(model: torch.nn.Module, x: Image):
    # evaluate the `model` on 8 rotated versions of the input image `x`
    model.eval()
    
    wrmup = model(torch.randn(1, 3, 80, 80).to(device))
    del wrmup
    
    #x = resize1(pad(x))
    
    print()
    print('##########################################################################################')
    header = 'angle |  ' + '  '.join(["{:6d}".format(d) for d in range(15)])
    print(header)
    with torch.no_grad():
        for r in range(8):
            #print(np.min(x), np.max(x), x.shape)
            intimg = x.astype(np.uint8)
            #print(type(intimg))
            #print(np.min(intimg), np.max(intimg), intimg.shape)
            # go from 3,80,80 -> 80,80,3
            #intimg2 = intimg.permute((1, 2, 0)).contiguous() # can't permute numpy arrays
            intimg2 = intimg.transpose(1, 2, 0)
            #print(np.min(intimg2), np.max(intimg2), intimg2.shape)
            img = Image.fromarray(intimg2)
            #print(np.min(img), np.max(img))
            rotimg = img.rotate(r*45., Image.BILINEAR)
            #print(rotimg.shape)
            x_transformed = totensor(rotimg).reshape(-1, 3, 80, 80)
            x_transformed = x_transformed.to(device)

            y = model(x_transformed)
            y = y.to('cpu').numpy().squeeze()
            
            angle = r * 45
            print("{:3d}:{}".format(angle, y))
    print('##########################################################################################')
    print()

In [12]:
# evaluate the model
#test_model(model, x)

The output of the model is already almost invariant.
However, we still observe small fluctuations in the outputs.

This is because the model contains some operations which might break equivariance.
For instance, every convolution includes a padding of $2$ pixels per side. This is adds information about the actual orientation of the grid where the image/feature map is sampled because the padding is not rotated with the image. 

During training, the model will observe rotated patterns and will learn to ignore the noise coming from the padding.

So, let's train the model now.
The model is exactly the same used to train a normal *PyTorch* architecture:

In [13]:
# Prep the training dataset
train_transform = Compose([
    #pad,
    #resize1,
    #RandomRotation(180, resample=Image.BILINEAR, expand=False),
    #resize2,
    #ToTensor()
])

#data_train = DOTARotDataset(mode='train', transform=train_transform)
#train_loader = torch.utils.data.DataLoader(data_train, batch_size=8, shuffle=True)
#print(len(data_train), len(train_loader))

In [14]:
def get_fresh_training_data(n):
    data_train = DOTARotDataset(mode='train', transform=train_transform, max_num_examples=n)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=8, shuffle=True, drop_last=True)
    #print(len(data_train), len(train_loader))
    return train_loader

In [15]:
# Prep the testing dataset
test_transform = Compose([
    #pad,
    #ToTensor()
])

data_test = DOTARotDataset(mode='test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=8, shuffle=False, drop_last=True)
print(len(data_test), len(test_loader))

self.num_samples 28838
28838 3604


In [16]:
# mini = 99999
# maxi = 0
# for m in data_train: # data_test
#     if m[1] < mini:
#         mini = m[1]
#     if m[1] > maxi:
#         maxi = m[1]
# print(mini, maxi, "<--- should be 0 14 instead of 1 15")

In [17]:
# Initialize the model
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)

In [18]:
def save_the_model(model, epoch, train_acc, train_loss, test_acc, test_loss):
    if not os.path.exists("models"):
        os.mkdir("models")
#    if not os.path.exists("models/model_dota.pth"):
    torch.save(model.state_dict(), 
               "models/model_dota_"+str(epoch)+"_"+str(round(train_acc,4))+"_"+str(round(train_loss,4))+"_"+str(round(test_acc,4))+"_"+str(round(test_loss,4))+".pth")
#    print('saved')
#    else:
#         model.load_state_dict(torch.load("models/model_dota.pth"))
#         print('loaded')

In [19]:
def save_the_results(epoch, train_accuracy, train_loss, test_accuracy, test_loss):
    add_headers = False
    if not os.path.exists("results"):
        os.mkdir("results")
    if not os.path.exists("results/model_dota.csv"):
        add_headers = True
    # writing to csv file
    with open("results/model_dota.csv", 'a') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile)

        if add_headers:
            csvwriter.writerow(['Epoch', 'Train Accuracy', 'Train Loss', 'Test Accuracy', 'Test Loss'])

        # writing the data rows
        csvwriter.writerow([epoch, train_accuracy, train_loss, test_accuracy, test_loss])

In [20]:
# # get initial performance with random weights
# test_total = 0
# test_correct = 0
# test_loss = 0
# with torch.no_grad():
#     model.eval()
    
#     #for i, (x, t) in enumerate(test_loader):
#     with tqdm(test_loader, unit="batch") as tepoch:
#         for x, t in tepoch:
#             tepoch.set_description(f"Epoch {-1}")
            
#             #if i%1000==0:
#             #    print(i, "/", len(test_loader))

#             x = x.to(device)
#             t = t.to(device)

#             y = model(x)

#             _, prediction = torch.max(y.data, 1)
#             if prediction.shape[0] != t.shape[0]:
#                 print(t)
#                 t = t[:-(t.shape[0]-prediction.shape[0])]
#                 print(t)
#             test_total += t.shape[0]
#             test_correct += (prediction == t).sum().item()

#             loss = loss_function(y, t)
#             test_loss += loss

# test_accuracy = test_correct/test_total*100.
    
# print(f"test accuracy: {test_accuracy}")
# print(f"test loss: {test_loss.item()}")

In [21]:
# save_the_results(0, 0, test_accuracy, test_loss.item())
# save_the_model(model, 0, 0, 0, test_accuracy, test_loss.item())

In [22]:
#train_loader = get_fresh_training_data(30000)

self.num_samples 98891


In [23]:
samples_per_class = 500
start_epoch = 60
max_epochs = 100

for epoch in range(start_epoch, max_epochs):
    
    print('starting epoch', epoch)
    
    ########################################
    # TRAIN
    ########################################
    train_total = 0
    train_correct = 0
    train_loss = 0
    model.train()
    
    #train_loader = get_fresh_training_data(samples_per_class)
    #samples_per_class += 1000
    
    #for i, (x, t) in enumerate(train_loader):
    with tqdm(train_loader, unit="batch") as tepoch:
        for x, t in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            #if i%5000==0:
            #    print(i, "/", len(train_loader))

            optimizer.zero_grad()

            x = x.to(device)
            t = t.to(device)

            y = model(x)

            _, prediction = torch.max(y.data, 1)
            # sometimes at the end of an epoch, prediction.shape can be < t.shape
            if prediction.shape[0] != t.shape[0]:
                print(t)
                t = t[:-(t.shape[0]-prediction.shape[0])]
                print(t)
            train_total += t.shape[0]
            train_correct += (prediction == t).sum().item()

            loss = loss_function(y, t)
            train_loss += loss

            loss.backward()

            optimizer.step()

    ########################################
    # TEST
    ########################################
    test_total = 0
    test_correct = 0
    test_loss = 0
    
    with torch.no_grad():
        model.eval()
        
        #for i, (x, t) in enumerate(test_loader):
        with tqdm(test_loader, unit="batch") as tepoch:
            for x, t in tepoch:
                tepoch.set_description(f"Epoch {epoch}")
                
                #if i%5000==0:
                #    print(i, "/", len(test_loader))

                x = x.to(device)
                t = t.to(device)

                y = model(x)

                _, prediction = torch.max(y.data, 1)
                if prediction.shape[0] != t.shape[0]:
                    print(t)
                    t = t[:-(t.shape[0]-prediction.shape[0])]
                    print(t)
                test_total += t.shape[0]
                test_correct += (prediction == t).sum().item()

                loss = loss_function(y, t)
                test_loss += loss

    train_accuracy = train_correct/train_total*100.
    test_accuracy = test_correct/test_total*100.

    #print(f"epoch {epoch} | train accuracy: {train_accuracy}")
    #print(f"epoch {epoch} | train loss: {train_loss.item()}")
    #print(f"epoch {epoch} | test accuracy: {test_accuracy}")
    #print(f"epoch {epoch} | test loss: {test_loss.item()}")

    tepoch.set_postfix({"train_accuracy":train_accuracy, 
                        "train_loss":train_loss.item(),
                        "test_accuracy":test_accuracy,
                        "test_loss":test_loss.item()})
    tepoch.close()
    time.sleep(0.5)
    
    save_the_results(epoch, train_accuracy, train_loss.item(), test_accuracy, test_loss.item())
    save_the_model(model, epoch, train_accuracy, train_loss.item(), test_accuracy, test_loss.item())

starting epoch 60


Epoch 60:   6%|▌         | 762/12361 [03:29<53:15,  3.63batch/s]


KeyboardInterrupt: 

In [ ]:
# train_accuracy = train_correct/train_total*100.
# print(train_correct, train_total, train_accuracy)
#save_the_results(epoch, train_accuracy, train_loss.item(), test_accuracy, test_loss.item())
#save_the_model(model, epoch, train_accuracy, train_loss.item(), test_accuracy, test_loss.item())

In [ ]:
# print(prediction.shape, t.shape)
# print(prediction)
# print(t)
# print(t[:-1])
# print(y.shape)
# print(torch.max(y.data, 1)) # 1 is the dimension

In [ ]:
# epoch -1 | test accuracy: 10.011096469935502
# epoch -1 | test loss: 2109988.5

# epoch 0 | train accuracy: 52.91684784257414
# epoch 0 | train loss: 42925.43359375
# epoch 0 | test accuracy: 18.846660656078782
# epoch 0 | test loss: 201328.765625

In [ ]:
#data_test = DOTARotDataset(mode='test')

In [ ]:
# retrieve the first image from the test set
x, y = next(iter(data_test))

# evaluate the model
test_model(model, x)